# Names, title, etc

In [ ]:
DONT_RUN_LONG_CELLS = True
from pandas import read_csv
kickstarter_filename = 'kickstarter_data_full.csv'
kickstarter_filename_features = 'kickstarter_data_with_features.csv'

ks_data = read_csv(kickstarter_filename)
ks_feat_data = read_csv(kickstarter_filename_features)
data_list = [('ks_data', ks_data), ('ks_feat_data', ks_feat_data)]
# print columns not present in the other dataset
ks_data_columns = set(ks_data.columns)
ks_feat_data_columns = set(ks_feat_data.columns)

ks_data_not_in_feat = ks_data_columns - ks_feat_data_columns
ks_feat_data_not_in_ks = ks_feat_data_columns - ks_data_columns

print(f'Columns in ks_data not in ks_feat_data: {ks_data_not_in_feat}')
print(f'Columns in ks_feat_data not in ks_data: {ks_feat_data_not_in_ks}')

common_columns = ks_data_columns.intersection(ks_feat_data_columns)

# of common columns, compare the values and see if they match
for column in common_columns:
    ks_data_values = ks_data[column].unique()
    ks_feat_data_values = ks_feat_data[column].unique()
    if len(ks_data_values) != len(ks_feat_data_values):
        print(f'Column {column} has different number of unique values: {len(ks_data_values)} vs {len(ks_feat_data_values)}')
    else:
        pass

## So we know ks_data is the dataset to use as it has more data, more unique data

In [ ]:
data_list = [('ks_data', ks_data), ('ks_feat_data', ks_feat_data)]
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import set_option


def data_info(_data, show_scatter=False):
    print(f'Data head: {_data.head(5)}')
    print(f'Null values: {_data.isnull().sum()}')
    print(f'Data Shape: {_data.shape[0]} rows and {_data.shape[1]} columns')
    print(f'Columns: {list(_data.columns)}')
    for column in _data.columns:
        if len(_data[column].unique()) < 10:
            print(f'{column} unique values: {_data[column].unique()}')
        else:
            percent_unique = len(_data[column].unique()) / _data.shape[0] * 100
            print(f'{column} % unique values: {percent_unique}')
    print(_data.describe())
    _data.hist()
    plt.tight_layout()
    plt.show()
    plt.figure()  # new plot
    plt.tight_layout()

    # Only show for floats and ints

    float_columns = _data.select_dtypes(include=['float64']).columns
    int_columns = _data.select_dtypes(include=['int64']).columns
    # Combine float and int columns

    numeric_columns = float_columns.append(int_columns)
    # Calculate correlation matrix
    corMat = _data[numeric_columns].corr(method='pearson')

    print(corMat)
    ## plot correlation matrix as a heat map
    sns.heatmap(corMat, square=True)
    plt.yticks(rotation=0)
    plt.xticks(rotation=90)
    plt.title(f"CORRELATION MATRIX USING HEAT MAP")
    plt.show()

    ## scatter plot of all _data
    plt.figure()
    # # The output overlaps itself, resize it to display better (w padding)
    if show_scatter:
        try:
            scatter_matrix(_data)
            plt.tight_layout(pad=0.1)
            plt.show()
        except:
            return


def data_info2(_date):
    set_option('display.max_columns', None)
    if not isinstance(_date, list):
        if not isinstance(_date, tuple):
            _date = ('', _date)
        _date = [_date]
    for name, data in _date:
        print(f'{name} data')
        print(data.info())
        print(data.head(5))
        for column in data.columns:
            highlight_column = 'profile'
            if column == highlight_column:
                # print 2 rows of values completely
                row1 = data.iloc[0][highlight_column]
                row2 = data.iloc[1][highlight_column]
                print(f'Row 1: {row1}')
                print(f'Row 2: {row2}')
            if len(data[column].unique()) < 10:
                print(f'{column} unique values: {data[column].unique()}')
            else:
                percent_unique = len(data[column].unique()) / data.shape[0] * 100
                print(f'{column} % unique values: {percent_unique}')
        break

In [ ]:
data_info(ks_data)

## Theres a lot to clean up here. First we need to categorize each column, then determine what we need to change

In [ ]:
columns_to_drop = ['Unnamed: 0', 'id', 'photo', 'name', 'blurb', 'slug', 'currency_symbol', 'currency_trailing_code', 'static_usd_rate', 'creator', 'profile', 'friends', 'is_backing', 'permissions', 'name_len', 'blurb_len', 'urls', 'source_url', 'location', 'is_starred', 'create_to_launch']
float_columns = ['goal', 'pledged', 'usd_pledged']
int_columns = ['backers_count', 'name_len_clean', 'blurb_len_clean',  'launch_to_deadline', 'launch_to_state_change', 'create_to_launch_days', 'launch_to_deadline_days', 'launch_to_state_change_days', ]
datetime_columns = ['deadline', 'state_changed_at', 'created_at', 'launched_at']
date_int_columns = ['deadline_month', 'deadline_day', 'deadline_hr', 'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_yr', 'created_at_month', 'created_at_day', 'created_at_hr', 'launched_at_month', 'launched_at_day', 'launched_at_hr', 'state_changed_at_hr', 'created_at_yr', 'launched_at_yr']
category_columns = ['state', 'currency', 'staff_pick', 'category', 'deadline_weekday', 'state_changed_at_weekday', 'created_at_weekday', 'launched_at_weekday', 'deadline_yr', 'country']
boolean_columns = ['disable_communication', 'spotlight', 'SuccessfulBool', 'USorGB', 'TOPCOUNTRY', 'LaunchedTuesday', 'DeadlineWeekend']

# This is just to categorize everything and make sure I'm not missing anything
temp_data = ks_data.copy()
data_info2(temp_data)
temp_data = temp_data.drop(columns=columns_to_drop)
temp_data = temp_data.drop(columns=datetime_columns)
temp_data = temp_data.drop(columns=category_columns)
temp_data = temp_data.drop(columns=boolean_columns)
temp_data = temp_data.drop(columns=float_columns)
temp_data = temp_data.drop(columns=int_columns)
temp_data = temp_data.drop(columns=date_int_columns)
print('\n' * 3)
print(f'Lets see what we have left')
data_info2(('temp_data', temp_data))

In [ ]:
kickstarter = ks_data.copy()
#kickstarter = kickstarter.dropna()
kickstarter = kickstarter.drop(columns=columns_to_drop)
# Specifically examine the problematic columns
print("NaN in name_len_clean:", kickstarter['name_len_clean'].isna().sum())
print("NaN in blurb_len_clean:", kickstarter['blurb_len_clean'].isna().sum())

print(kickstarter['category'].unique())
# total number of records
print(kickstarter.shape)
# Number of na's in 'category'
category_nas = kickstarter['category'].isna().sum()
total_rows = kickstarter.shape[0]
print(f'Category has {(category_nas / total_rows * 100):.2f}% as "nan"')
temp_data = ks_data.copy()
# Drop the 10 rows which have _len_clean as nan
kickstarter = kickstarter.dropna(subset=['name_len_clean', 'blurb_len_clean'])
print("NaN in name_len_clean:", kickstarter['name_len_clean'].isna().sum())
print("NaN in blurb_len_clean:", kickstarter['blurb_len_clean'].isna().sum())

kickstarter['name_len_clean'] = kickstarter['name_len_clean'].astype(int)
kickstarter['blurb_len_clean'] = kickstarter['blurb_len_clean'].astype(int)

# Replace 'nan' in category with 'None' (string not object), then do one-hot encoding
kickstarter['category'] = kickstarter['category'].fillna("None")
print(kickstarter['category'].unique())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def encode_categorical_features(df: pd.DataFrame) -> tuple:
    """Encode all categorical features in the dataframe"""
    # Create a copy to avoid modifying the original
    df = df.copy()

    # Get all categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    print(f"Categorical columns: {categorical_columns}")

    # Handle boolean columns separately
    boolean_columns = ['staff_pick']
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].replace({False: 0, True: 1})
            print(f"Converted boolean column: {col}")

    # Columns to one-hot encode
    columns_to_encode = [
        'category',
        'deadline_weekday',
        'created_at_weekday',
        'launched_at_weekday'
    ]

    # Filter out columns that aren't in the dataframe
    columns_to_encode = [col for col in columns_to_encode if col in df.columns]

    # Remove 'state_changed_at_weekday' column if it exists
    if 'state_changed_at_weekday' in df.columns:
        df = df.drop(columns='state_changed_at_weekday')
        print("Dropped 'state_changed_at_weekday' column")

    # One-hot encode the selected columns
    all_encoded_dfs = []

    for col in columns_to_encode:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = encoder.fit_transform(df[[col]])

        # Create meaningful column names
        encoded_cols = [f"{col}_{c}" for c in encoder.categories_[0]]
        encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)

        # Add to list of encoded dataframes
        all_encoded_dfs.append(encoded_df)

        # Drop the original column
        print(f'Dropping original column: {col}')
        df = df.drop(columns=col)
        print(f"Encoded column: {col} → {len(encoded_cols)} features")

    # Combine all dataframes
    result = pd.concat([df] + all_encoded_dfs, axis=1)

    # Create a list of all categorical columns (original and encoded)
    all_categorical_columns = []
    for col in categorical_columns:
        if col in result.columns:
            all_categorical_columns.append(col)
        else:
            # Add the encoded column names
            all_categorical_columns.extend([c for c in result.columns if c.startswith(f"{col}_")])

    print(f"Total categorical columns after encoding: {len(all_categorical_columns)}")

    return result, all_categorical_columns

# Example usage
kickstarter, category_columns = encode_categorical_features(kickstarter)

## Now we know what is in the data, let's look at some of the features and see if we can find any interesting patterns.

In [ ]:
# For each feature, lets see a histogram of the values and a boxplot of the values
def plot_feature_distribution(data, feature):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(data[feature], bins=30, kde=True)
    plt.title(f'{feature} Distribution')
    plt.xlabel(feature)
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    sns.boxplot(x=data[feature])
    plt.title(f'{feature} Boxplot')
    plt.xlabel(feature)

    plt.tight_layout()
    plt.show()

columns_to_show = float_columns + int_columns

for feature in kickstarter.columns:
    if feature in columns_to_show and not DONT_RUN_LONG_CELLS:
        plot_feature_distribution(kickstarter, feature)

In [ ]:
columns_to_standardize = [ # Roughly normal
    'name_len_clean', 'blurb_len_clean',
    'deadline_month', 'deadline_day', 'deadline_hr',
    'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_hr',
    'created_at_month', 'created_at_day', 'created_at_hr',
    'launched_at_month', 'launched_at_day', 'launched_at_hr'
]

columns_to_log_transform = [ # Heavily right skewed
    'goal', 'pledged', 'usd_pledged', 'backers_count',
    'create_to_launch_days'
]

columns_to_normalize = [ # Different scales
    'launch_to_deadline_days', 'launch_to_state_change_days'
]

columns_to_drop2 = [
    # Temporal columns represented by better features
    'deadline', 'state_changed_at', 'created_at', 'launched_at',
    'launch_to_deadline', 'launch_to_state_change',

    # Duplicate/redundant information
    'deadline_yr', 'state_changed_at_yr', 'created_at_yr', 'launched_at_yr',

    # Categorical with many unique values, better represented by other features
    'country', 'currency',

    # Low variance or binary columns that might be redundant
    'disable_communication', 'spotlight',
    'USorGB', 'TOPCOUNTRY', 'LaunchedTuesday', 'DeadlineWeekend'
]

# Drop the columns we don't want
kickstarter = kickstarter.drop(columns=columns_to_drop2)
print(kickstarter.columns)

In [ ]:
for column in columns_to_log_transform:
    print(f"NaN values in '{column}': {kickstarter[column].isna().sum()}")

print(kickstarter.shape[0])
kickstarter = kickstarter.dropna(subset=columns_to_log_transform)
print(kickstarter.shape[0])

# Standardize the columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for column in columns_to_standardize:
    kickstarter[column] = scaler.fit_transform(kickstarter[[column]])

# Log transform the columns
from sklearn.preprocessing import FunctionTransformer
import numpy as np
log_transformer = FunctionTransformer(np.log1p, validate=True)
for column in columns_to_log_transform:
    kickstarter[column] = log_transformer.fit_transform(kickstarter[[column]])
# Normalize the columns
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler()
for column in columns_to_normalize:
    kickstarter[column] = normalizer.fit_transform(kickstarter[[column]])

In [ ]:

# Check the data again
data_info(kickstarter)

## We need to make a 'success' classifier, so lets take a look at the definition of success in the data

In [ ]:
# Compare 'state' and 'SuccessfulBool' to see if they are the same
print(ks_data['state'].unique())
print(ks_data['SuccessfulBool'].unique())
print(ks_data['state'].value_counts())
print(ks_data['SuccessfulBool'].value_counts())
# Make a table where the rows are 0/1 for SuccessfulBool and the columns are the states
print(ks_data.groupby(['SuccessfulBool', 'state']).size().unstack())

# Show the distribution of 'pledged/goal' to see if we can make a column called 'got_funded'
temp_data = ks_data.copy()

# confirm they're both numbers
print(temp_data['pledged'].dtype)
print(temp_data['goal'].dtype)

# show the count where pledged > goal
print(f'{temp_data[temp_data['pledged'] > temp_data['goal']].shape[0]} projects got funded')

# Create got_funded column and show the histogram
temp_data['got_funded'] = temp_data['pledged'] / temp_data['goal']

# Describe the column
print(temp_data['got_funded'].describe())

# Get count of SuccessfulBool
print(temp_data['SuccessfulBool'].value_counts())
# Shows the same thing

# So SuccessfulBool doesn't capture why it failed but does capture if it was successful and closely matches if the project was funded, but not how much it was funded.

In [ ]:
# using kickstarter data, show some good plots of features vs SuccessfulBool

def plot_feature_vs_success(data, feature):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(data[data['SuccessfulBool'] == 1][feature], bins=30, kde=True, color='green', label='Successful')
    sns.histplot(data[data['SuccessfulBool'] == 0][feature], bins=30, kde=True, color='red', label='Failed')
    plt.title(f'{feature} vs SuccessfulBool')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.legend()

    plt.subplot(1, 2, 2)
    sns.boxplot(x=data['SuccessfulBool'], y=data[feature])
    plt.title(f'{feature} vs SuccessfulBool')
    plt.xlabel('SuccessfulBool')
    plt.ylabel(feature)

    plt.tight_layout()
    plt.show()

# Show the features that are not categorical
for feature in kickstarter.columns:
    if DONT_RUN_LONG_CELLS:
        break
    if feature not in ['state', 'SuccessfulBool', 'category', 'currency', 'staff_pick']:
        plot_feature_vs_success(kickstarter, feature)

### Because `state` captures the same information as `SuccessfulBool`, we can drop it
### and because `got_funded` and `pledged` offer information obtained at the end of the campaign, we should drop them as well

In [ ]:
kickstarter = kickstarter.drop(columns=['state', 'pledged'])

In [ ]:
# Determine the avg goal for each category, save to a hashmap
category_goals = {}

temp_data = ks_data.copy()
unique_categories = ks_data['category'].unique()
unique_categories = np.delete(unique_categories, 2)
unique_categories_list = unique_categories.tolist()
unique_categories_list.append('None')
unique_categories = np.array(unique_categories_list)

print(unique_categories)

for category in unique_categories:
    # get the avg goal for the category
    avg_goal = ks_data[ks_data['category'] == category]['goal'].mean()
    # store as regular string and regular float
    category_goals[category] = avg_goal

print(category_goals)

# Now make a new column, goal_percentile
def make_percentile(goal_amount: float, category: str) -> float:
    """Return what percentile the goal is compared to average for its category"""
    avg_for_category = category_goals.get(category, 0)
    if avg_for_category == 0:
        return 0

    return goal_amount / avg_for_category * 100

# Then apply the function to create a new column
kickstarter['goal_percentile'] = ks_data.groupby('category')['goal'].transform(
    lambda x: x.rank(pct=True)
) * 100  # Scale to 0-100

# Show the distribution of goal_percentile as a box plot
def plot_goal_percentile_boxplot(data):
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=data['goal_percentile'])
    plt.title('Goal Percentile Boxplot')
    plt.xlabel('Goal Percentile')
    plt.show()

plot_goal_percentile_boxplot(kickstarter)

In [ ]:
# make a new column funding_needed_per_day which is the goal / launch_to_deadline

# Drop all rows with launch_to_deadline_days = 0
kickstarter = kickstarter[kickstarter['launch_to_deadline_days'] != 0]

kickstarter['funding_needed_per_day'] = kickstarter['goal'] / kickstarter['launch_to_deadline_days']
# show the distribution of funding_needed_per_day
def plot_funding_needed_per_day_boxplot(data):
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=data['funding_needed_per_day'])
    plt.title('Funding Needed Per Day Boxplot')
    plt.xlabel('Funding Needed Per Day')
    plt.show()

plot_funding_needed_per_day_boxplot(kickstarter)
import numpy as np

# 1. Check where the infinities or extreme values are
print("Infinities in funding_needed_per_day:", np.isinf(kickstarter['funding_needed_per_day']).sum())
print("NaNs in funding_needed_per_day:", np.isnan(kickstarter['funding_needed_per_day']).sum())
print("Max value in funding_needed_per_day:", kickstarter['funding_needed_per_day'].max())

# 2. Check the source columns - likely a division by zero
print("Zeros in launch_to_deadline_days:", (kickstarter['launch_to_deadline_days'] == 0).sum())
# Normalize it
kickstarter['funding_needed_per_day'] = normalizer.fit_transform(kickstarter[['funding_needed_per_day']])

plot_funding_needed_per_day_boxplot(kickstarter)


In [ ]:
print(kickstarter.columns)

In [ ]:
from sklearn.model_selection import cross_val_score
from itertools import combinations
from sklearn.tree import DecisionTreeRegressor
all_features = kickstarter.columns
best_score = 0
best_features = None
X = kickstarter[all_features]
Y1 = kickstarter['SuccessfulBool']
X = X.drop(columns=['SuccessfulBool'])

for i in range(1, len(all_features) + 1):
    for feature_combo in combinations(all_features, i):
        features = list(feature_combo)
        X = kickstarter[features]
        scores = cross_val_score(DecisionTreeRegressor(), X, Y1, cv=5, scoring='r2')
        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
            best_features = features

print(f"Best features: {best_features} with R² score: {best_score:.4f}")